In [1]:
from langchain_openai import ChatOpenAI
import os
import openai
import getpass
import tiktoken
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
#==========================================================================================#
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.retrievers import BM25Retriever
from langchain.retrievers import EnsembleRetriever


In [2]:
tokenizer = tiktoken.get_encoding("cl100k_base")

def tiktoken_len(text):
    tokens = tokenizer.encode(text)
    return len(tokens)

In [3]:
from langchain_community.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='../data/dur.csv')
data = loader.load()

In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 250, chunk_overlap = 100, length_function=tiktoken_len)
texts = text_splitter.split_documents(data)

In [5]:
model_name = "jhgan/ko-sbert-nli"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

c:\Users\Admin\anaconda3\envs\sba\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\Admin\anaconda3\envs\sba\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
c:\Users\Admin\anaconda3\envs\sba\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [6]:
#save to disk
emed_db = Chroma.from_documents(texts, hf,persist_directory="../db/dur_jhgan_250")

In [7]:
#벡터db의 데이터가 in-memory가 아니라 persistent storage인 disk에 저장되게 선언
emed_db.persist()
emed_db = None

In [8]:
# load from disk
emed_db = Chroma(persist_directory="../db/dur_jhgan_250",embedding_function=hf)

In [17]:
openai = ChatOpenAI(model = "gpt-3.5-turbo",temperature=0)

vector_retriever = emed_db.as_retriever(search_type="similarity", search_kwargs={'k':3})

# Initialize the BM25 retriever
bm25_retriever = BM25Retriever.from_documents(data)
bm25_retriever.k =  3

ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever, vector_retriever],
                                       weights=[0.7, 0.3])

In [18]:
ensemble_retriever.invoke("타세놀")

[Document(page_content='상세정보: 연령 금기. 아래의 약은 특정 연령의 복용이 제한된다.\n제품명: 타세놀\n성분명: acetaminophen(encapsulated)\n업체명: 부광약품(주)\n특정 연령: 12세 미만\n금기 사유: 소아 및 고령자(노인)는 최소 필요량을 복용하고 이상반응에 유의. 과도한 체온강하, 허탈, 사지냉각 등이 나타날 수 있음', metadata={'source': '../data/dur.csv', 'row': 3}),
 Document(page_content='상세정보: 임부 금기. 아래의 약은 임산부의 복용이 불가하다.\n제품명: 콜드엔정\n성분명: pseudoephedrine\n업체명: (주)대웅제약\n금기 등급: 2\n금기 사유: 임부에 대한 안전성 미확립.', metadata={'source': '../data/dur.csv', 'row': 444287}),
 Document(page_content='제품명B: 삼아돔페리돈말레산염정\n성분명A: itraconazole\n성분명B: domperidone maleate (as domperidone)\n업체명A: (주)티디에스팜\n업체명B: 삼아제약(주)\n금기 사유: QTc 연장 효과 증대로 심각한 위험(Torsade de Pointes, 심각한 심실성 부정맥 포함) 가능성 강력한 CYP억제제가 돔페리돈의 대사를 감소시켜, Domperidone의 영향(이상반응 포함) 증대 가능성, Itraconazole 투여 중 및 종료 후 2주 간 해당 성분 투여 금기', metadata={'row': 222055, 'source': '../data/dur.csv'}),
 Document(page_content='제품명B: 삼아돔페리돈말레산염정\n성분명A: itraconazole\n성분명B: domperidone maleate (as domperidone)\n업체명A: (주)디에이치피코리아\n업체명B: 삼아제약(주)\n금기 사유: QTc 연장 효과 증대로 심각한 위험(Tor

In [19]:
query = "타세놀 노인이 먹어도 돼?"
docs = ensemble_retriever.get_relevant_documents(query)
docs

[Document(page_content='상세정보: 연령 금기. 아래의 약은 특정 연령의 복용이 제한된다.\n제품명: 타세놀\n성분명: acetaminophen(encapsulated)\n업체명: 부광약품(주)\n특정 연령: 12세 미만\n금기 사유: 소아 및 고령자(노인)는 최소 필요량을 복용하고 이상반응에 유의. 과도한 체온강하, 허탈, 사지냉각 등이 나타날 수 있음', metadata={'source': '../data/dur.csv', 'row': 3}),
 Document(page_content='상세정보: 연령 금기. 아래의 약은 특정 연령의 복용이 제한된다.\n제품명: 타다프리정\n성분명: tadalafil\n업체명: 알보젠코리아(주)\n특정 연령: 18세 미만\n금기 사유: nan', metadata={'row': 3878, 'source': '../data/dur.csv'}),
 Document(page_content='상세정보: 임부 금기. 아래의 약은 임산부의 복용이 불가하다.\n제품명: 콜드엔정\n성분명: pseudoephedrine\n업체명: (주)대웅제약\n금기 등급: 2\n금기 사유: 임부에 대한 안전성 미확립.', metadata={'source': '../data/dur.csv', 'row': 444287})]

In [20]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human","{input}"),
    ]
)

history_aware_retriever = create_history_aware_retriever(
    openai, ensemble_retriever, contextualize_q_prompt
)

#답변 생성
qa_system_prompt = """You are an assistant for question-answering tasks. \
ONLY USE the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(openai, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever,question_answer_chain)

# chat history 관리
store = {}

def get_session_history(session_id : str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key= "input",
    history_messages_key= "chat_history",
    output_messages_key= "answer",
)

In [21]:
id = "0"
query1 = "타세놀 노인이 먹어도 돼?"
conversational_rag_chain.invoke(
    {"input": query1},
    config={
        "configurable": {"session_id":id}
    },
)#["answer"]

{'input': '타세놀 노인이 먹어도 돼?',
 'chat_history': [],
 'context': [Document(page_content='상세정보: 연령 금기. 아래의 약은 특정 연령의 복용이 제한된다.\n제품명: 타세놀\n성분명: acetaminophen(encapsulated)\n업체명: 부광약품(주)\n특정 연령: 12세 미만\n금기 사유: 소아 및 고령자(노인)는 최소 필요량을 복용하고 이상반응에 유의. 과도한 체온강하, 허탈, 사지냉각 등이 나타날 수 있음', metadata={'source': '../data/dur.csv', 'row': 3}),
  Document(page_content='상세정보: 연령 금기. 아래의 약은 특정 연령의 복용이 제한된다.\n제품명: 타다프리정\n성분명: tadalafil\n업체명: 알보젠코리아(주)\n특정 연령: 18세 미만\n금기 사유: nan', metadata={'row': 3878, 'source': '../data/dur.csv'}),
  Document(page_content='상세정보: 임부 금기. 아래의 약은 임산부의 복용이 불가하다.\n제품명: 콜드엔정\n성분명: pseudoephedrine\n업체명: (주)대웅제약\n금기 등급: 2\n금기 사유: 임부에 대한 안전성 미확립.', metadata={'source': '../data/dur.csv', 'row': 444287})],
 'answer': '노인은 타세놀을 최소 필요량만 복용하고 이상반응에 유의해야 합니다. 과도한 체온강하, 허탈, 사지냉각 등의 이상반응이 나타날 수 있으니 주의가 필요합니다. 타세놀은 12세 미만의 소아에게 금기되어 있습니다.'}